In [1]:
from pulp import *
import pandas as pd

In [2]:
probA = LpProblem("Workforce", LpMinimize)

In [3]:
month = list(range(1,7))

required = {1:100,
            2:100,
            3:115,
            4:125,
            5:140,
            6:150
           }


In [4]:
remaining = LpVariable.dicts("r", month, lowBound = 0, cat = "Continuous")
training = LpVariable.dicts("t", month, lowBound = 0, cat = "Continuous")
untrained = LpVariable.dicts("ut", month, lowBound = 0, cat = "Continuous")
trained = LpVariable.dicts("td", month, lowBound = 0, cat = "Continuous")
layoff = LpVariable.dicts("l", month, lowBound = 0, cat = "Continuous")

In [5]:
probA += lpSum((training[i] + untrained[i])*3000 + trained[i]*3300 for i in range(1,7))

In [6]:
probA += trained[1] == 0, "Trained Programmer 1"

for i in range(2,7):
    probA += lpSum(0.9 * training[j] for j in range(1,i))\
    == trained[i], "Trained Programmer %s"%i

In [7]:
probA += remaining[1] == 145, "Total Programmer 1"

for i in range(2,7):
    probA += lpSum(remaining[i-1]- layoff [i-1]- (remaining[i-1] - trained[i-1] - training[i-1] - layoff[i-1]) * 0.05)  \
    == remaining[i],"Total Programmer %s"%i

In [8]:
for i in range(1,7):
    probA += lpSum(trained[i]+training[i]+untrained[i])\
    == remaining[i], "Untrained Programmer %s"%i

In [9]:
for i in range(1,7):
    probA += lpSum(untrained[i]+trained[i]*1.2)\
    >= required[i], "Required Programmer %s"%i

In [10]:
probA

Workforce:
MINIMIZE
3000*t_1 + 3000*t_2 + 3000*t_3 + 3000*t_4 + 3000*t_5 + 3000*t_6 + 3300*td_1 + 3300*td_2 + 3300*td_3 + 3300*td_4 + 3300*td_5 + 3300*td_6 + 3000*ut_1 + 3000*ut_2 + 3000*ut_3 + 3000*ut_4 + 3000*ut_5 + 3000*ut_6 + 0
SUBJECT TO
Trained_Programmer_1: td_1 = 0

Trained_Programmer_2: 0.9 t_1 - td_2 = 0

Trained_Programmer_3: 0.9 t_1 + 0.9 t_2 - td_3 = 0

Trained_Programmer_4: 0.9 t_1 + 0.9 t_2 + 0.9 t_3 - td_4 = 0

Trained_Programmer_5: 0.9 t_1 + 0.9 t_2 + 0.9 t_3 + 0.9 t_4 - td_5 = 0

Trained_Programmer_6: 0.9 t_1 + 0.9 t_2 + 0.9 t_3 + 0.9 t_4 + 0.9 t_5 - td_6
 = 0

Total_Programmer_1: r_1 = 145

Total_Programmer_2: - 0.95 l_1 + 0.95 r_1 - r_2 + 0.05 t_1 + 0.05 td_1 = 0

Total_Programmer_3: - 0.95 l_2 + 0.95 r_2 - r_3 + 0.05 t_2 + 0.05 td_2 = 0

Total_Programmer_4: - 0.95 l_3 + 0.95 r_3 - r_4 + 0.05 t_3 + 0.05 td_3 = 0

Total_Programmer_5: - 0.95 l_4 + 0.95 r_4 - r_5 + 0.05 t_4 + 0.05 td_4 = 0

Total_Programmer_6: - 0.95 l_5 + 0.95 r_5 - r_6 + 0.05 t_5 + 0.05 td_5 = 0

Unt

In [11]:
probA.writeLP("problem_1.lp")
probA.solve()
print("Status:", LpStatus[probA.status])

Status: Optimal


In [12]:
for v in probA.variables():
    print(v.name, "=", v.varValue)

l_1 = 11.147463
l_2 = 0.0
l_3 = 0.0
l_4 = 0.0
l_5 = 0.0
r_1 = 145.0
r_2 = 129.40991
r_3 = 126.83991
r_4 = 125.54544
r_5 = 125.19233
r_6 = 125.18011
t_1 = 45.0
t_2 = 37.50991
t_3 = 26.691693
t_4 = 20.201733
t_5 = 8.484932
t_6 = 0.0
td_1 = 0.0
td_2 = 40.5
td_3 = 74.258919
td_4 = 98.281443
td_5 = 116.463
td_6 = 124.09944
ut_1 = 100.0
ut_2 = 51.4
ut_3 = 25.889297
ut_4 = 7.0622685
ut_5 = 0.24439637
ut_6 = 1.0806698


In [13]:
value(probA.objective)

2467583.94561